# Setup


In [1]:
import numba
import numpy as np

Create a small array of random number such that it can be used to test the algorithms to make sure they work.


In [4]:
np.random.seed(42)
test_array = np.random.randint(1, 100_000, 100, dtype=np.int32)
test_array[0:10]

array([15796,   861, 76821, 54887,  6266, 82387, 37195, 87499, 44132,
       60264], dtype=int32)

# Casual Use


## Implementation #1

Edited slightly so it can take in an optional parameter.


In [8]:
import random


def digit_sum(n):
    """Calculate the sum of digits of a number"""
    return sum(int(digit) for digit in str(n))


def find_difference(numbers=None):
    if not numbers:
        # Generate list of 1 million random integers
        numbers = [random.randint(1, 100000) for _ in range(1000000)]

    # Initialize variables for min and max numbers with digit sum 30
    min_num = float("inf")  # Initialize to positive infinity
    max_num = float("-inf")  # Initialize to negative infinity

    # Find numbers whose digits sum to 30
    for num in numbers:
        if digit_sum(num) == 30:
            min_num = min(min_num, num)
            max_num = max(max_num, num)

    # Check if we found any numbers with digit sum 30
    if min_num == float("inf") or max_num == float("-inf"):
        return "No numbers found with digit sum of 30"

    return max_num - min_num


def casual_implementation_1(numbers=None):
    return find_difference(numbers)

In [9]:
find_difference(test_array.tolist())

62910

Benchmark the code: due to slowness, only do 10 trials for this implementation.


In [12]:
%%timeit -n 1 -r 10

find_difference()

629 ms ± 3.73 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)


## Implementation #2
